So my own SGD doesn't seem to be converging towards what the analytical solution gives. The squared loss function is convex due to X.TX always being positive and semi definite, so it should with enough epochs converge towards analytical. Sklearn is able to reach it with a lot of iterations. Though they might be using some fancy stuff for quicker convergence. Might test with momentum too though.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

In [60]:
import numpy as np
from modelling import ols,ridge
from modelling.sgd import SGD_optimizer
from data.create_dataset import *
from visualization.visualize import *
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
X, z = create_dataset('../data/raw/SRTM_data_Norway_1.tif',degree=4)
X_train, X_test, z_train, z_test = train_test_split(X,z, test_size=0.2)

In [4]:
beta_ols = ols.fit_beta(X_train,z_train)
beta_ols

array([[ 1161.60416278],
       [  731.60705663],
       [  406.09728099],
       [-1494.27341142],
       [-2223.48693796],
       [ -788.65010401],
       [ 1696.80944233],
       [ 3307.8261914 ],
       [  817.63621267],
       [ 1138.02663525],
       [ -894.20916574],
       [ -717.52395709],
       [-1619.21466643],
       [  755.01568821],
       [ -795.77629324]])

Above is truth or analytical solution. SGD should reach with enough epochs.

In [49]:
Xscaler = StandardScaler().fit(X_train)
zscaler = StandardScaler().fit(z_train)

#X_train_scl = Xscaler.transform(X_train)
#z_train_scl = zscaler.transform(z_train)
X_train_scl = X_train - np.mean(X_train,axis=0)
z_train_scl = z_train - np.mean(z_train,axis=0)

In [52]:
sgdreg = SGDRegressor( random_state=10, fit_intercept = False, max_iter = 100000, penalty=None,alpha=1,learning_rate = 'constant', eta0 = 0.001,tol=None)
sgdreg = sgdreg.fit(X_train_scl,z_train_scl.ravel())
print(sgdreg.coef_.T)
print(sgdreg.intercept_.T)

[    0.           677.88172141   366.72735468 -1325.92684199
 -2122.07632564  -678.30074455  1486.13118085  3183.40664783
   713.47476958  1007.59515263  -804.24161291  -664.53285218
 -1576.6274202    794.63845807  -741.94133067]
[0.]


In [7]:
sgdreg2 = SGDRegressor( random_state=10, fit_intercept = True, max_iter = 100000, penalty=None,alpha=1,learning_rate = 'constant', eta0 = 0.001,tol=None)
sgdreg2 = sgdreg2.fit(X_train[:,1:],z_train.ravel())
print(sgdreg2.coef_.T)
print(sgdreg2.intercept_.T)

[  678.20515241   367.02555952 -1326.31244476 -2122.43873302
  -678.64138267  1486.78080198  3183.88749225   713.99335689
  1008.1678578   -804.41847264  -664.70927845 -1576.76259406
   794.45534669  -742.06405811]
[1167.7579321]


So after having gone through whole SKlearn SGD source code I've figured out that they actually calculate intercept and weights separately. It seems they initially do the X@beta - z , add this to the intercept, then completes the 1/n(X.T(X@beta-z)). However I can't see that they actually multiply by two for some reason. Even though the gradient should have a 2 in front. But updating intercept afterwards is actually exactly the same as only keeping it in the design matrix. Because they updates the weight with $X\times c$ where c is prediction-truth. This same scalar is directly added to the intercept, which is the same as first scaling it with 1, which is in the first column of X, and then add it. Simply keeping 1's i n the design matrix should result in the same.

What needs to happen with Ridge, is that first column must be pulled out, data must be centered, and then during cost gradient, pull out X@beta -z and return as well, or do similar as sklearn, simply do prediction and subtract z in a function, which we call update or something, continue running this one further for rest of betas, but add to intercept. So I need prediction at some point. But sum of all? Because I'm operating with batches, and not one sample at a time like SKlearn.

In [11]:
(X_train.T @ (X_train @ beta_ols-z_train)).shape

(8000, 1)

In [20]:
np.set_printoptions(suppress=True)

In [58]:
sgdown = ols.fit_beta_sgd(X_train_scl,z_train_scl,batch_size=1,n_epochs=100000, lr=0.001)
sgdown

test
8000


array([[   -0.204324  ],
       [  677.84250857],
       [  366.21086142],
       [-1326.4926003 ],
       [-2121.39395492],
       [ -678.33446858],
       [ 1487.04854988],
       [ 3183.95371078],
       [  712.98282652],
       [ 1007.29483881],
       [ -804.14021266],
       [ -664.86039468],
       [-1576.16098222],
       [  795.11487984],
       [ -742.5917441 ]])

Ok good, so it does converge correctly, with same epochs and lr as sklearn. Only problem is the intercept now. How to calculate? I simply must rescale afterwards. So if I'd want to get the same intercept as SKlearn?

In [70]:
sgd = SGD_optimizer(batch_size = 1, n_epochs = 1,use_momentum= False, gamma = 0.5,regularization = None, lmb = 1, lr=0.001)
beta_class = sgd.fit(X_train_scl,z_train_scl)
beta_class

In [71]:
beta_class.beta

array([[ 0.        ],
       [36.44979845],
       [-4.72354289],
       [34.90751015],
       [28.55293119],
       [-6.09129011],
       [29.05666631],
       [30.78957577],
       [23.05939687],
       [-6.70535627],
       [23.46525216],
       [27.97684843],
       [25.38675151],
       [19.30913268],
       [-7.06176838]])